In [1]:
'''
This notebook submits the jobs to aws batch using imported generated Latin hypercube sampling inputs generated from another notebook.

It does not do a good job of monitoring the batch jobs, so your best bet is to log into the aws console and monitor the jobs there. 
I'm working on it, but it's not quite there yet.

There's a variable set to True or False towards the end that you'll want to start with as False and change to True when jobs are done.
After that, it will download all of the outfile.lis.csv files and then create a single .tgz file out of all of them in the outputs directory.
'''

import boto3
import matrices.LHS_1000  # make sure the size is correct!


matrix = matrices.LHS_1000.matrix  # here too!
print(len(matrix))

1000


In [2]:
size = len(matrix)
size

1000

In [3]:
# experiment name/number
exp = 'soybeans'

In [4]:
client = boto3.client('batch')
client

In [5]:
jobs = []
jobs

[]

In [6]:
jobs_uri = "s3://daycent-jobs/jobs/{}_{}/jobs".format(exp, size)
jobs_uri
# s3://daycent-jobs/jobs/corn_1000/diffs/diff_0.tgz

's3://daycent-jobs/jobs/soybeans_1000/jobs'

In [7]:
# s3://daycent-jobs/jobs/corn_1000/diffs/diff_0.tgz

In [8]:
%%time

# submit the jobs to aws batch 

for i in range(size):  # size
    job_name = 'test_{}_{}_{}'.format(exp, size, i)
    response = client.submit_job(
        jobDefinition='test-job-def',
        jobName=job_name,
        jobQueue='queue',
        containerOverrides={
          'command': [ 
              "-s","soybeans",
              "-n","soybeans",
              "-e","soybeans_prev",
              "-f","s3://daycent-jobs/jobs/soybeans/soybeans_prev.bin",  # bin to extend from
              "-l","yes",  # run list100 after
              "-i","s3://daycent-jobs/jobs/soybeans/soybeans.tgz",  # full input directory
              "-j", jobs_uri,  # job s3 bucket
              # "-o","yes",  # if you want to download the complete output directory rather than just the output file
              "-d","s3://daycent-jobs/jobs/{}_{}/diffs/diff_{}.tgz".format(exp, size, i),  # make sure this makes sense
              "-r","{}".format(i),
              #"-c", "yes" # capture stderr and stdout to daycent.log.txt and list100.log.txt
          ],  
        },
    )
    jobs.append(response)
    # print(response)
print("Submitted {} jobs.".format(len(jobs)))

Submitted 1000 jobs.
CPU times: user 4.43 s, sys: 211 ms, total: 4.64 s
Wall time: 2min 15s


In [9]:
print("num jobs is {}".format(len(jobs)))

num jobs is 1000


In [10]:
job_listing = client.list_jobs(
    jobQueue = 'queue',
    jobStatus='RUNNABLE',
)
job_listing
#print("Job list gave {} jobs".format(len(job_listing)))

{'ResponseMetadata': {'RequestId': 'd8b8e744-540f-4637-8ca4-890b3977223f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 13 Oct 2022 13:34:09 GMT',
   'content-type': 'application/json',
   'content-length': '215868',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd8b8e744-540f-4637-8ca4-890b3977223f',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'Z8gIxEw3IAMFjzg=',
   'access-control-expose-headers': 'X-amzn-errortype,X-amzn-requestid,X-amzn-errormessage,X-amzn-trace-id,X-amz-apigw-id,date',
   'x-amzn-trace-id': 'Root=1-634813d1-566d3a151f075c190d9c4962'},
  'RetryAttempts': 0},
 'jobSummaryList': [{'jobArn': 'arn:aws:batch:us-east-1:712153133523:job/07af3857-3bc9-40bf-af1e-aee70405ea98',
   'jobId': '07af3857-3bc9-40bf-af1e-aee70405ea98',
   'jobName': 'test_soybeans_1000_13',
   'createdAt': 1665667916372,
   'status': 'RUNNABLE'},
  {'jobArn': 'arn:aws:batch:us-east-1:712153133523:job/09378c1c-0d5c-4cc4-b966-d48d5a3cd417',
   'jobId': '09378c1c-0d

In [11]:
ids = []
for job in jobs:
    ids.append(job['jobId'])
print(len(ids))

1000


In [12]:
jobstuff = client.describe_jobs(
    jobs=ids[-100:]
)
for job in jobstuff['jobs']:
    print("name: {}, status: {}".format(job['jobName'], job['status']))

name: test_soybeans_1000_943, status: RUNNABLE
name: test_soybeans_1000_910, status: RUNNABLE
name: test_soybeans_1000_978, status: RUNNABLE
name: test_soybeans_1000_975, status: RUNNABLE
name: test_soybeans_1000_917, status: RUNNABLE
name: test_soybeans_1000_982, status: RUNNABLE
name: test_soybeans_1000_950, status: RUNNABLE
name: test_soybeans_1000_941, status: RUNNABLE
name: test_soybeans_1000_963, status: RUNNABLE
name: test_soybeans_1000_956, status: RUNNABLE
name: test_soybeans_1000_973, status: RUNNABLE
name: test_soybeans_1000_957, status: RUNNABLE
name: test_soybeans_1000_930, status: RUNNABLE
name: test_soybeans_1000_992, status: RUNNABLE
name: test_soybeans_1000_938, status: RUNNABLE
name: test_soybeans_1000_995, status: RUNNABLE
name: test_soybeans_1000_990, status: RUNNABLE
name: test_soybeans_1000_997, status: RUNNABLE
name: test_soybeans_1000_924, status: RUNNABLE
name: test_soybeans_1000_964, status: RUNNABLE
name: test_soybeans_1000_914, status: RUNNABLE
name: test_so

In [17]:
# randomly samples a subset of jobs and prints their status to the output every few seconds for a few iterations
# clears the previous output before printing the new one
# you can just run this cell over and over again to get an idea of the jobs, not perfect, but it's somewhere to start

import time
import random
from IPython.display import clear_output

iterations = 40  # number of times to pull jobs and get their status
num_jobs = 75 # number of jobs to randomly select from to view
sleep_time = 5  # number of seconds to sleep between job polling

for i in range(iterations):
    done = True
    jobstuff = client.describe_jobs(
        jobs=list(random.sample(ids, num_jobs))
    )
    # jobstuff
    clear_output(wait=True)
    output = ""
    for stuff in jobstuff['jobs']:
        output = output + "i={} : {} : {}\n".format(i, stuff['jobName'], stuff['status'])
    print(output, end='\r')
    time.sleep(sleep_time)
print('done')

i=39 : test_soybeans_1000_707 : SUCCEEDED
i=39 : test_soybeans_1000_103 : SUCCEEDED
i=39 : test_soybeans_1000_530 : SUCCEEDED
i=39 : test_soybeans_1000_34 : SUCCEEDED
i=39 : test_soybeans_1000_950 : SUCCEEDED
i=39 : test_soybeans_1000_38 : SUCCEEDED
i=39 : test_soybeans_1000_657 : SUCCEEDED
i=39 : test_soybeans_1000_345 : SUCCEEDED
i=39 : test_soybeans_1000_809 : SUCCEEDED
i=39 : test_soybeans_1000_769 : SUCCEEDED
i=39 : test_soybeans_1000_547 : SUCCEEDED
i=39 : test_soybeans_1000_267 : SUCCEEDED
i=39 : test_soybeans_1000_53 : SUCCEEDED
i=39 : test_soybeans_1000_16 : SUCCEEDED
i=39 : test_soybeans_1000_573 : SUCCEEDED
i=39 : test_soybeans_1000_644 : SUCCEEDED
i=39 : test_soybeans_1000_819 : SUCCEEDED
i=39 : test_soybeans_1000_358 : SUCCEEDED
i=39 : test_soybeans_1000_622 : SUCCEEDED
i=39 : test_soybeans_1000_218 : SUCCEEDED
i=39 : test_soybeans_1000_863 : SUCCEEDED
i=39 : test_soybeans_1000_253 : SUCCEEDED
i=39 : test_soybeans_1000_277 : SUCCEEDED
i=39 : test_soybeans_1000_392 : SUCCEE

In [18]:
import os

print(os.getcwd())
if not os.path.isdir('outputs/{}_{}'.format(exp, size)):
    os.makedirs('outputs/{}_{}'.format(exp, size))

/home/jovyan/accelerator/newdata/soy


In [19]:
# flip this to True when the jobs are finished running, or put in False to pause here while they're still running
jobs_done = True
jobs_done

True

In [20]:
%%time

# copy all the outfile.lis.csv files back to the outputs directory named to correspond to their job number

import subprocess

# just holding things up here to wait for the jobs to finish since I can't check them all because aws doesn't like checking on more than
# 100 jobs at a time. Change to jobs_done True to continue
if jobs_done: 
    
    for i in range(len(matrix)):
        # grab the csv files and stick them somewhere
        # cmd should look like:
        # aws s3 cp s3://daycent-jobs/jobs/exp1000/jobs/{}/outfile.lis.csv ./outputs/exp1000/outfile_{}.lis.csv
        subprocess.call(['/home/jovyan/aws-cli/v2/current/bin/aws', 's3', 'cp', 
                         '{}/{}/outfile.lis.csv'.format(jobs_uri, i), 
                         './outputs/{}_{}/outfile_{}.list.csv'.format(exp, size, i)])
print("sure done, why not.")

sure done, why not.
CPU times: user 401 ms, sys: 4.46 s, total: 4.86 s
Wall time: 19min 50s


In [21]:
%%time

# create a single tgz file containing all of the outfile_{job}.lis.csv files

# just holding things up here to wait for the jobs to finish since I can't check them all because aws doesn't like checking on more than
# 100 jobs at a time. Change jobs_done to True to continue
if jobs_done:
    import os

    os.chdir('outputs')
    print(os.getcwd())

    # tgz them all up
    subprocess.call(['tar', 'czvf', '{}_{}.tgz'.format(exp, size), '{}_{}'.format(exp, size)])
    print('done tgzing them')

/home/jovyan/accelerator/newdata/soy/outputs
done tgzing them
CPU times: user 4.96 ms, sys: 4.53 ms, total: 9.49 ms
Wall time: 384 ms
